In [ ]:
# 소벨 연산자 

import cv2 as cv

# 이미지 로드
img=cv.imread('./opencv_data/soccer.jpg')
# gray 이미지로 변환
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)

# 소벨 연산자를 적용
grad_x=cv.Sobel(gray,cv.CV_32F,1,0,ksize=3)
grad_y=cv.Sobel(gray,cv.CV_32F,0,1,ksize=3)

# 절대값을 취해 양수 영상으로 변환
sobel_x = cv.convertScaleAbs(grad_x)
sobel_y = cv.convertScaleAbs(grad_y)

edge_strength = cv.addWeighted(sobel_x,0.5,sobel_y,0.5,0)   # 에지 강도 계산

cv.imshow('Original',gray)
cv.imshow('sobelx',sobel_x)
cv.imshow('sobely',sobel_y)
cv.imshow('edge strength',edge_strength)

cv.waitKey()
cv.destroyAllWindows()

In [17]:
# 캐니 에지 실험하기

import cv2 as cv

img=cv.imread('./opencv_data/soccer.jpg')   # 영상 읽기

gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)

canny1=cv.Canny(gray,100,150)   # Tlow=50, Thigh=150으로 설정
canny2=cv.Canny(gray,100,200)   # Tlow=100, Thigh=200으로 설정
canny3=cv.Canny(gray,50,150)
canny4=cv.Canny(gray,50,200)

canny1_small=cv.resize(canny1, dsize=(0,0), fx=0.5, fy=0.5)
canny2_small=cv.resize(canny2, dsize=(0,0), fx=0.5, fy=0.5)
canny3_small=cv.resize(canny3, dsize=(0,0), fx=0.5, fy=0.5)
canny4_small=cv.resize(canny4, dsize=(0,0), fx=0.5, fy=0.5)

cv.imshow('Original',gray)
cv.putText(canny1_small, 'low=100, high=150', (10,30), cv.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (255,255,255), 2)
cv.imshow('Canny1',canny1_small)
cv.putText(canny2_small, 'low=100, high=200', (10,30), cv.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (255,255,255), 2)
cv.imshow('Canny2',canny2_small)
cv.putText(canny3_small, 'low=50, high=150', (10,30), cv.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (255,255,255), 2)
cv.imshow('Canny3',canny3_small)
cv.putText(canny4_small, 'low=50, high=200', (10,30), cv.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (255,255,255), 2)
cv.imshow('Canny4',canny4_small)

cv.waitKey()
cv.destroyAllWindows()

In [30]:
# 경계선 찾기

import cv2 as cv
import numpy as np

img=cv.imread('./opencv_data/soccer.jpg')   # 영상 읽기
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)   # 그레이스케일로 변환
canny = cv.Canny(gray,100,200)   # 캐니 에지 검출

# 경계선 찾기 : findContours() 함수
contour, hierarchy= cv.findContours(canny, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
# print(contour)

lcontour = [] # 엣지 라인이 들어갈 변수
for i in range(len(contour)):
    if contour[i].shape[0] > 100:
        lcontour.append(contour[i])

cv.drawContours(img,lcontour,-1,(0,255,0),3)   # 경계선 그리기

cv.imshow('Original with Contour', img)
cv.imshow('Canny',canny)


cv.waitKey()
cv.destroyAllWindows()

In [27]:
len(lcontour)

106

In [2]:
# 허프 변환
import cv2 as cv 

img=cv.imread('./opencv_data/apples.jpg')
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)

apples=cv.HoughCircles(gray,cv.HOUGH_GRADIENT,1,200,param1=150,param2=20,minRadius=50,maxRadius=120)

for i in apples[0]: 
    cv.circle(img,(int(i[0]),int(i[1])),int(i[2]),(255,0,0),2)

cv.imshow('Apple detection',img)  

cv.waitKey()
cv.destroyAllWindows()

In [ ]:
%pip install scikit-image

In [6]:
# SLIC 알고리즘으로 입력 영상을 슈퍼 화소 분할하기

import skimage
import numpy as np
import cv2 as cv

img=skimage.data.coffee()
cv.imshow('Coffee image',cv.cvtColor(img,cv.COLOR_RGB2BGR))

# 영역 분할 SLIC 알고리즘 적용 - slic()
slic1=skimage.segmentation.slic(img,compactness=20,n_segments=600) # 영역 분할
sp_img1=skimage.segmentation.mark_boundaries(img,slic1) # 영역분할 표시
sp_img1=np.uint8(sp_img1*255.0) # 이미지 표시를 위해서  0~255 양수로 변환

slic2=skimage.segmentation.slic(img,compactness=45,n_segments=600)
sp_img2=skimage.segmentation.mark_boundaries(img,slic2)
sp_img2=np.uint8(sp_img2*255.0)

cv.imshow('Super pixels (compact 20)',cv.cvtColor(sp_img1,cv.COLOR_RGB2BGR))
cv.imshow('Super pixels (compact 40)',cv.cvtColor(sp_img2,cv.COLOR_RGB2BGR))

cv.waitKey()
cv.destroyAllWindows()

In [9]:
# 정규화 절단 알고리즘으로 영역 분할하기

import skimage
import numpy as np
import cv2 as cv
import time
from skimage import graph

# 커피 이미지 데이터 로드
coffee=skimage.data.coffee()

# 시간
start=time.time()
# SLIC
slic=skimage.segmentation.slic(coffee,compactness=20,n_segments=600,start_label=1)
# 유사도
g=graph.rag_mean_color(coffee,slic,mode='similarity') 
ncut=graph.cut_normalized(slic,g)   # 정규화 절단
print(coffee.shape,' Coffee 영상을 분할하는데 ',time.time()-start,'초 소요')

print(g)
print(len(ncut))

marking=skimage.segmentation.mark_boundaries(coffee,ncut)
ncut_coffee=np.uint8(marking*255.0)

cv.imshow('Normalized cut',cv.cvtColor(ncut_coffee,cv.COLOR_RGB2BGR))  

cv.waitKey()
cv.destroyAllWindows()

(400, 600, 3)  Coffee 영상을 분할하는데  2.243699789047241 초 소요
RAG with 546 nodes and 2099 edges
400


In [ ]:
# GrabCut 알고리즘으로 영상 분할하기

import cv2 as cv 
import numpy as np

img=cv.imread('./opencv_data/soccer.jpg')   # 영상 읽기
img_show=np.copy(img)      # 붓 칠을 디스플레이할 목적의 영상

mask=np.zeros((img.shape[0],img.shape[1]),np.uint8) 
mask[:,:]=cv.GC_PR_BGD      # 모든 화소를 배경일 것 같음으로 초기화
                            # cv.GC_FGD : 확실히 물체
                            # cv.GC_BGD : 확실히 배경
                            # cv.GC_PR_FGD : 물체인 것 같음 
                            # cv.GC_PR_BGD : 배경인 것 같음

BrushSiz=9            # 붓의 크기
LColor,RColor=(255,0,0),(0,0,255)   # 파란색(물체)과 빨간색(배경)

def painting(event,x,y,flags,param):
    if event==cv.EVENT_LBUTTONDOWN:   
        cv.circle(img_show,(x,y),BrushSiz,LColor,-1)   # 왼쪽 버튼 클릭하면 파란색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_FGD,-1)
    elif event==cv.EVENT_RBUTTONDOWN: 
        cv.circle(img_show,(x,y),BrushSiz,RColor,-1)   # 오른쪽 버튼 클릭하면 빨간색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_BGD,-1)
    elif event==cv.EVENT_MOUSEMOVE and flags==cv.EVENT_FLAG_LBUTTON:
        cv.circle(img_show,(x,y),BrushSiz,LColor,-1)# 왼쪽 버튼 클릭하고 이동하면 파란색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_FGD,-1)
    elif event==cv.EVENT_MOUSEMOVE and flags==cv.EVENT_FLAG_RBUTTON:
        cv.circle(img_show,(x,y),BrushSiz,RColor,-1)   # 오른쪽 버튼 클릭하고 이동하면 빨간색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_BGD,-1)

    cv.imshow('Painting',img_show)
    
cv.namedWindow('Painting')
cv.setMouseCallback('Painting',painting)

while(True):            # 붓 칠을 끝내려면 'q' 키를 누름
    if cv.waitKey(1)==ord('q'): 
        break

# 여기부터 GrabCut 적용하는 코드
background=np.zeros((1,65),np.float64)   # 배경 히스토그램 0으로 초기화
foreground=np.zeros((1,65),np.float64)   # 물체 히스토그램 0으로 초기화

cv.grabCut(img,mask,None,background,foreground,5,cv.GC_INIT_WITH_MASK)
mask2=np.where((mask==cv.GC_BGD)|(mask==cv.GC_PR_BGD),0,1).astype('uint8')
grab=img*mask2[:,:,np.newaxis]
cv.imshow('Grab cut image',grab)  

cv.waitKey()
cv.destroyAllWindows()

In [19]:
# 영역 특징
import skimage
import numpy as np
import cv2 as cv

orig=skimage.data.horse()
img=255-np.uint8(orig)*255
cv.imshow('Horse',img)

# 에지 경계 추출
contours,hierarchy=cv.findContours(img,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_NONE)


img2 = cv.cvtColor(img,cv.COLOR_GRAY2BGR)      # 그레이 스케일 영상을 컬러로 변환
cv.drawContours(img2,contours,-1,(255,0,255),2)
cv.imshow('Horse with contour',img2)

contour = contours[0]

m = cv.moments(contour)            # 몇 가지 특징 
area = cv.contourArea(contour)
cx,cy = m['m10']/m['m00'],m['m01']/m['m00']
perimeter = cv.arcLength(contour,True)
roundness = (4.0*np.pi*area)/(perimeter*perimeter)
print('면적=',area,'\n중점=(',cx,',',cy,')','\n둘레=',perimeter,'\n둥근 정도=',roundness)

img3 = cv.cvtColor(img,cv.COLOR_GRAY2BGR)      # 컬러 디스플레이용 영상

contour_approx = cv.approxPolyDP(contour,8,True)   # 직선 근사
cv.drawContours(img3,[contour_approx],-1,(0,255,0),2)

hull = cv.convexHull(contour)         # 볼록 헐
hull = hull.reshape(1,hull.shape[0],hull.shape[2])
cv.drawContours(img3,hull,-1,(0,0,255),2)

cv.imshow('Horse with line segments and convex hull',img3)

cv.waitKey()
cv.destroyAllWindows()

면적= 42390.0 
중점=( 187.72464024534088 , 144.43640402610677 ) 
둘레= 2296.7291333675385 
둥근 정도= 0.1009842680321435


In [ ]:
## 해리스 특징점
import cv2 as cv
import numpy as np

img=np.array([[0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,0,1,0,0,0,0,0,0],
              [0,0,0,1,1,0,0,0,0,0],
              [0,0,0,1,1,1,0,0,0,0],
              [0,0,0,1,1,1,1,0,0,0],
              [0,0,0,1,1,1,1,1,0,0],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0]],dtype=np.float32)

ux=np.array([[-1,0,1]])
uy=np.array([-1,0,1]).transpose()
k=cv.getGaussianKernel(3,1)
g=np.outer(k,k.transpose())

dy=cv.filter2D(img,cv.CV_32F,uy)
dx=cv.filter2D(img,cv.CV_32F,ux)
dyy=dy*dy
dxx=dx*dx
dyx=dy*dx
gdyy=cv.filter2D(dyy,cv.CV_32F,g)
gdxx=cv.filter2D(dxx,cv.CV_32F,g)
gdyx=cv.filter2D(dyx,cv.CV_32F,g)
C=(gdyy*gdxx-gdyx*gdyx)-0.04*(gdyy+gdxx)*(gdyy+gdxx)

for j in range(1,C.shape[0]-1):      # 비최대 억제
    for i in range(1,C.shape[1]-1):
        if C[j,i]>0.1 and sum(sum(C[j,i]>C[j-1:j+2,i-1:i+2]))==8:
            img[j,i]=9         # 특징점을 원본 영상에 9로 표시
                
np.set_printoptions(precision=2)
print('dy :',dy) 
print('dx :',dx) 
print('dyy :',dyy) 
print('dxx :',dxx) 
print('dyx :',dyx) 
print('gdyy :',gdyy) # 6
print('gdxx :',gdxx) # 7
print('gdyx :',gdyx) # 8
print('C :',C)               # 특징 가능성 맵 
print('img :',img)               # 특징점을 9로 표시한 원본 영상 

popping=np.zeros([160,160],np.uint8)   # 화소 확인 가능하게 16배로 확대
for j in range(0,160):
    for i in range(0,160):
        popping[j,i]=np.uint8((C[j//16,i//16]+0.06)*700)  

cv.imshow('Image Display2',popping)    
cv.waitKey()
cv.destroyAllWindows()

dy : [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0.  0. -1. -1. -1. -1.  0.  0.  0.]
 [ 0.  0.  0. -1. -1. -1. -1. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
dx : [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1. -1. -1.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0. -1. -1.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0. -1. -1.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0. -1. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
dyy : [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 